In [3]:
#read in data
train=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\projects\train_oSwQCTC\train.csv')
train.count()

550068

In [4]:
#take a peak at the data
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [5]:
#read in data
test=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\projects\test_HujdGe7\test.csv')
test.count()

233599

In [6]:
#checking for missing data
from pyspark.sql.functions import col,sum
train.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in train.columns)).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [7]:
from pyspark.sql.functions import col,sum
test.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in test.columns)).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|             72344|            162562|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+



In [8]:
#Product ID by itself is of not much advantage so we find the avg price of it.
mean= train.groupBy(train.Product_ID).mean('Purchase')

In [9]:
#Join will add the 2 column so cleaning up the names
mean=mean.withColumnRenamed('avg(Purchase)', 'Avg')
mean=mean.withColumnRenamed('Product_ID', 'Dup')

In [10]:
#New feature for avg spend of the user
purchasehistory= train.groupBy(train.User_ID).mean('Purchase')
#purchasehistory.show()
purchasehistory=purchasehistory.withColumnRenamed('avg(Purchase)', 'Avg_spend')
purchasehistory=purchasehistory.withColumnRenamed('User_ID', 'User_Dup')


In [11]:
#the test set has product ID which the train set doesnt have
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.count()

46

In [12]:
# New features of avg price of each product and avg spend of each user
train=train.join(mean,train.Product_ID==mean.Dup).join(purchasehistory,train.User_ID==purchasehistory.User_Dup)

train=train.drop('Product_Category_2','Product_Category_3','Dup','User_Dup')
train.show(3)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+--------+------------------+-----------------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Purchase|               Avg|        Avg_spend|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+--------+------------------+-----------------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|    8370|11870.863436123347|9545.514285714286|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|   15200|16304.030981067126|9545.514285714286|
|1000001| P00087842|     F|0-17|        10|            A|                         2|             0|                12|    1422|1237.8921568627452|9545.514285714286|
+-------+-

In [13]:
#repeating it for the test dataset, notice that the join 'left' will add Null values if the product ID is not present
test=test.join(mean,test.Product_ID==mean.Dup, how='left').join(purchasehistory,test.User_ID==purchasehistory.User_Dup,how='left')
test=test.drop('Product_Category_2','Product_Category_3','Dup','User_Dup')
test.show(3)


+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+-----------------+------------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|              Avg|         Avg_spend|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+-----------------+------------------+
|1000004| P00128942|     M|46-50|         7|            B|                         2|             1|                 1|15781.11858974359|14747.714285714286|
|1000009| P00113442|     M|26-35|        17|            C|                         0|             0|                 3|11746.66535433071|10243.086206896553|
|1000010| P00288442|     F|36-45|         1|            B|                        4+|             1|                 5|5731.338028169014| 9728.744394618834|
+-------+----------+------+-----+----------+-------------+

In [17]:
#recheck for missing values , 61 avg values need to be updated
from pyspark.sql.functions import col,sum
test.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in test.columns)).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---+---------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Avg|Avg_spend|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---+---------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0| 61|        0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---+---------+



In [14]:
#Null values of the Avg price of the Product ID that are not present in the train set are imputed
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=["Avg"], 
    outputCols=["Avg_imputed"]
)
df_change=imputer.fit(test).transform(test)


In [15]:
#drop and rename columns
tester=df_change.drop('Avg','Dup')
tester=tester.withColumnRenamed('Avg_imputed','Avg')

In [16]:
#recheck for missing values 
from pyspark.sql.functions import col,sum
tester.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in tester.columns)).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---------+---+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Avg_spend|Avg|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---------+---+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|        0|  0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+---------+---+



In [18]:
#convert categorical columns
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
categorical_columns = ['Age','Gender','Occupation','City_Category']

## build stages for string indexer and onehot encoding
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='stringindexed_' + c) for c in categorical_columns]
onehotencoder_stages = [OneHotEncoder(inputCol='stringindexed_' + c, outputCol='onehotencoded_' + c) for c in categorical_columns]
all_stages = stringindexer_stages + onehotencoder_stages

pipeline = Pipeline(stages=all_stages)

## fit pipeline model for train
df_coded = pipeline.fit(train).transform(train)




In [19]:
#selected features
selected_columns = ['onehotencoded_' + c for c in categorical_columns] + ['Avg','Marital_Status','Avg_spend']


In [20]:
#convert to features vector
vectorassembler = VectorAssembler(inputCols=selected_columns, outputCol='features')

# transform data
df_features = vectorassembler.transform(df_coded)

In [21]:
pipeline_mode = pipeline.fit(tester)

## transform data
test_coded = pipeline_mode.transform(tester)


In [22]:
test_features = vectorassembler.transform(test_coded)

In [23]:
# Linear regression 
import pyspark.ml.regression as rg
linalg= rg.LinearRegression(
    maxIter=10, 
    regParam=0.01, 
    labelCol='Purchase')



lrmodel = linalg.fit(df_features)
resulttrain= lrmodel.transform(df_features)

In [24]:
resulttest = lrmodel.transform(test_features)


In [25]:
#ensure all rows as per the original test dataset is present
resulttest.count()

233599

In [26]:
from pyspark.sql.functions import col,sum
submission =resulttest.select(col("User_ID"),col("Product_ID"),col("prediction"))
final=submission.withColumnRenamed('prediction','Purchase')
final.show(2)


+-------+----------+-----------------+
|User_ID|Product_ID|         Purchase|
+-------+----------+-----------------+
|1000004| P00128942|17939.08638407769|
|1000009| P00113442|11951.05908583899|
+-------+----------+-----------------+
only showing top 2 rows



In [27]:
#evaluatee on train
import pyspark.ml.evaluation as ev
evaluator = ev.RegressionEvaluator(
    predictionCol='prediction', 
    labelCol='Purchase')

In [28]:
print(evaluator.evaluate(resulttrain, 
    {evaluator.metricName: 'rmse'}))

2554.8615568235855


In [ ]:
#error is Rs 2554,